# Imports

In [3]:
from scipy.interpolate import LinearNDInterpolator
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numba import njit
from ROOT import TF1,TLorentzVector,TGenPhaseSpace,TFile,TH2D,TCanvas,TTree,TH1F,TGraphErrors,TH2F,gStyle,TText,TLatex,gPad, TGraph
import random
from scipy.interpolate import LinearNDInterpolator

Welcome to JupyROOT 6.26/00


# Numerical Constants

In [4]:
# Mass of Electron
# ---------------------------------------------
mE    = 0.000510999

# Mass of JPsi
# ---------------------------------------------
mJpsi = 3.0969

# Mass of Proton
# ---------------------------------------------
mP    = 0.938272

# Mass of Deuteron
# ---------------------------------------------
mD    = 1.8761358

# Fine Structure Constant
# ---------------------------------------------
alpha_em = 1.0/137.036

# Event Objects

In [5]:
# Phase Space Decay
# ---------------------------------------------
GenPhase = TGenPhaseSpace()

# 4-momentum of Particles
# ---------------------------------------------
eIn, hIn, eOut, hOut, q, VM, ePlus, eMinus = TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector()

# 4-momentum of Smeared Particles
# ---------------------------------------------
smear_eOut, smear_hOut, smear_ePlus, smear_eMinus, smear_q, smear_VM = TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector()

# Returned eventgen variables
# ---------------------------------------------
weight=np.array([0.0])
decay_weight=np.array([0.0])
psf=np.array([0.0])
flux=np.array([0.0])
acc_eOut=np.array([0.0])
acc_hOut=np.array([0.0])
acc_ePlus=np.array([0.0])
acc_eMinus=np.array([0.0])
gammaE_=np.array([0.0])
t_=np.array([0.0])
m_vm_=np.array([0.0])

## Functions

In [6]:
bremm = TF1("bremm","(4/3-4*x/3+x*x)/x",0.01,1)
bremm.SetNpx(1000)

def N_Bremmstrahlung(kmin,kmax,beamE,d,X0):
    ymin = kmin/beamE
    ymax = kmax/beamE
    y = bremm.GetRandom(ymin,ymax)
    flux = 0.5* d/X0 * (4.0 / 3.0 * np.log(ymax / ymin) - 4.0 / 3.0 * (ymax - ymin) + 1.0 / 2.0 * (ymax * ymax - ymin * ymin))
    return flux, y*beamE

@njit
def N_EquivalentPhotonApproximation(gammaE,beamE,q2max):
    # (https://lss.fnal.gov/archive/other/lpc-94-35.pdf)
    # Factors of mE^2 are needed atop the fractions 1/q2min and 1/q2max for units
    # I believe this is a mistake in the derivation
    y = gammaE/beamE
    q2min = mE**2 * y**2 / (1-y)
    return (alpha_em*y)/(4*np.pi)*((2*q2min+4*mE**2)/(q2min)*np.log(q2max/q2min)-mE**2/q2min+mE**2/q2max)


In [7]:
cthmax,cthmin=1,-1
def VirtualPhoton(kmin,kmax,beamE,mT):
    Pe  = random.uniform(beamE-kmax,beamE-kmin)
    cth = random.uniform(cthmin,cthmax)
    sth = np.sqrt(1-cth**2)
    phi = random.uniform(0,2*np.pi)
    eOut.SetXYZM(Pe * sth * np.cos(phi),Pe * sth * np.sin(phi),Pe * cth, mE)
    qq=eIn-eOut
    W2 = (qq+hIn)*(qq+hIn)
    
    if(W2 < mT**2):
        return 0, TLorentzVector()
    Q2 = -qq*qq
    flux = np.sqrt((hIn*q)**2 + Q2 * mT**2)/np.sqrt((eIn*hIn)**2 - mE**2 * mT**2)
    amp = (2.0 * Q2 - 4.0 * mE**2) / (Q2**2)
    phase = eOut.P()**2/(2.0*eOut.E()*(2.0*np.pi)**3)
    volume = 2.0 * np.pi * np.abs(kmax-kmin) * (cthmax-cthmin)
    y = (hIn * qq)/(hIn*eIn)
    gy = hIn.M() * np.sqrt(Q2)/(hIn * eIn)
    eps = (1.0 - y - 0.25 * gy * gy) / (1.0 - y + 0.5 * y * y + 0.25 * gy * gy)
    couple = 4.0 * np.pi * alpha_em
    return couple * flux * amp * phase * volume / (1.0 - eps),qq

# Nucleon Momentum within Deuteron

In [8]:
momentum_D = TF1("momentum_D","pow((1/(x*x+0.0456*0.0456)-1/(x*x+0.2719*0.2719)),2)*x*x",0,1)
momentum_D.SetNpx(1000)
def getNucleon():
    return momentum_D.GetRandom(),np.random.uniform(-1,1),np.random.uniform(0,2*np.pi)

## Cross Section Models

In [9]:
# -------------------------------
# 
# PROTON MODEL 
# (Pomeron LQCD)
# 
# -------------------------------
df=pd.read_csv("harrymodel/diff.csv",sep=",")
dfV = df.values
df_W = dfV[:,0]
df_Th = dfV[:,1]
df_DS = dfV[:,2]
interp_p = LinearNDInterpolator(list(zip(df_W,np.cos(np.pi*df_Th/180))),df_DS)

# Get dsdcth from interpolation
def dsdcth_p(W,cth):
    return interp_p(W,cth)

In [10]:
# -------------------------------
# 
# DEUTERON MODEL 
#
# -------------------------------

# Load the Model
df=pd.read_csv("crossSection.csv",sep="\t")
df=df.sort_values(by=['Beam Energy [GeV]','-t [GeV**2]'])
dfV=df.values
df_E=dfV[:,1]
df_T=dfV[:,2]
df_DSDT=dfV[:,3]
interp_d=LinearNDInterpolator(list(zip(df_E,df_T)),df_DSDT)

# Get tmin and tmax for each energy
tmtm = np.zeros((np.unique(df_E).size,3))
idxtmp=0
for tmpE in np.unique(df_E):
    tminTemp=df_T[np.where(df_E==tmpE)[0][0]]
    tmaxTemp=df_T[np.where(df_E==tmpE)[0][-1]]
    tmtm[idxtmp]=(tmpE,tminTemp,tmaxTemp)
    idxtmp+=1
    
@njit
def get_tmtm(gammaE):
    idx_of_E = np.abs(gammaE - tmtm[:,0]).argmin()
    return tmtm[idx_of_E,0],tmtm[idx_of_E,1],tmtm[idx_of_E,2]

# Get dsdt from interpolation
def dsdt_d(gammaE,t):
    return interp_d(gammaE,t)

In [11]:
N2g = 6.499e3
N3g = 2.894e3
v = 1/(16*np.pi)

def get_xsec(targetType,mT,modelType,gammaE,W,t,cth):
    
    
    # J = "Jacobian"
    # Since we generate gamma+(p,d) --> V + (p',d') with a uniform angular distribution (cth uniform)
    # we must multiply cross sections dsigma/dt by the appropriate dt/dcth Jacobian
    
    # PROTON CROSS SECTION
    # ------------------------------------
    if(targetType=='p' or (targetType=='d' and do_coherent==False)):
        if(modelType=='PomeronLQCD'):
            the_dsdcth = dsdcth_p(W,cth)
            return the_dsdcth

        elif(modelType=='23g'):
            x = (2*mP*mJpsi+mJpsi**2)/(W**2-mP**2)
            J = 2*pCM_Initial*pCM_Final/(2*np.pi)
            return J * (N2g * v * (1-x)**2*np.exp(1.13*t)/(mJpsi**2)+N3g*v*np.exp(1.13*t)/(mJpsi**4))

        
        
    # DEUTERON CROSS SECTION
    # ------------------------------------
    elif(targetType=='d' and modelType=='PomeronLQCD'):
        # Also store temporary "closest photon energy" from cross section data
        Egamma_temp,tmin,tmax = get_tmtm(gammaE)
        if(-t<tmin or -t>tmax):
            return -1 
        # Get dsdt of (gamma + p,d --> JPsi + p,d)
        # We use Egamma_temp as opposed to gammaE from Monte Carlo
        # because the interpolation for dsdt(Egamma,t) is guaranteed to work
        # when using an Egamma pulled specifically from the crossSection data
        the_dsdt=dsdt_d(Egamma_temp,-t)

        x = (2*mP*mJpsi+mJpsi**2)/(W**2-mP**2)
        J = 2*pCM_Initial*pCM_Final/(2*np.pi)
        return J * the_dsdt

## Acceptance

In [12]:
def acc_e(P):
    binx = acc.GetXaxis().FindBin(P.Theta()*180.0/np.pi)
    biny = acc.GetYaxis().FindBin(P.P())
    return acc.GetBinContent(binx,biny)

def acc_h(P):
    binx = acc.GetXaxis().FindBin(P.Theta()*180.0/np.pi)
    biny = acc.GetYaxis().FindBin(P.P())
    return acc.GetBinContent(binx,biny)

# Smearing

In [13]:
def smear_particle(P,particleType,det):
    p = P.P()
    th = P.Theta()
    phi= P.Phi()
    if(det=="SoLID"):
        if(particleType=="e+" or particleType=="e-"):
            dp = res_positron_p.GetBinContent(res_positron_p.FindBin(p,th*180.0/np.pi))/100.0
            dth = res_positron_th.GetBinContent(res_positron_th.FindBin(p,th*180.0/np.pi))/100.0
            dphi = res_positron_phi.GetBinContent(res_positron_phi.FindBin(p,th*180.0/np.pi))/100.0
        elif(particleType=="p" or particleType=="d"):
            dp = res_proton_p.GetBinContent(res_proton_p.FindBin(p,th*180.0/np.pi))/100.0
            dth = res_proton_th.GetBinContent(res_proton_th.FindBin(p,th*180.0/np.pi))/100.0
            dphi = res_proton_phi.GetBinContent(res_proton_phi.FindBin(p,th*180.0/np.pi))/100.0
    elif(det=="CLAS12"):
        dp = 0.01
        dth = 0.001
        dphi = 0.004
    pprime = p*np.random.normal(1,dp)
    thprime = th*np.random.normal(1,dth)
    phiprime = phi*np.random.normal(1,dphi)
    smear_P = TLorentzVector(pprime*np.sin(thprime)*np.cos(phiprime),
                            pprime*np.sin(thprime)*np.sin(phiprime),
                            pprime*np.cos(thprime),
                            np.sqrt(pprime**2+P.M2()))
    return smear_P

# ---> Event Parameters <---

In [14]:
# Monte Carlo Parameters
# ---------------------------------------------
events=5000000 #Number of events
processID=1 # 0 = photo, 1 = electro
modelType="PomeronLQCD"
do_acc = True
do_smear = do_acc & True

# Experimental Parameters
# ---------------------------------------------
beamE=11      #Electron beam energy
beamCurrent = 1.25 # MicroAmps
days=50        #Days of beamtime
fprefix="test_py"

# Detector Setup Parameters
# ---------------------------------------------
targetType="d"
do_coherent = True
d = 15           # Target Length [cm]
detector="SoLID"

# Kinematic Limits
# ---------------------------------------------
kmin=6      # Minimum photon energy
kmax=8.8    # Maximum photon energy
q2max=0.1  # Maximum Q2 of virtual photon (for quasi-real photoproduction)

In [16]:
# Filename attachment
# ---------------------------------------------
if(processID==0):
    processStr="photo"
else:
    processStr="electro"
if(do_coherent==True):
    processStr+="_coherent"
else:
    processStr+="_incoherent"

# Target Mass
# ---------------------------------------------
if(targetType=="p" or (do_coherent==False and targetType=="d")):
    mT = mP
elif(targetType=="d"):
    mT = mD
    
    
    
    
# Luminosity
# ---------------------------------------------
if(targetType=="p"):
    rho     = 0.071 # Density of Target g/cm3
    X0      = 890   # Radiation Length [cm]
    factor  = 1
if(targetType=="d"):
    rho     = 0.169 # Density of Target g/cm3
    X0      = 769.1 # Radiation Length [cm]
    factor = 1/2
lumi    = factor * beamCurrent * 1e-6/(1.6e-19) * d * rho * 6.02e23 * 1.0e-24 * 1.0e-9 * 3600.0 * 24 * days 





# Set Detector System
# ---------------------------------------------
if(detector=='SoLID'):
    facc = TFile("acceptance/acceptance_solid_JPsi_electron_target315_output.root","r")
    acc = facc.Get("acceptance_ThetaP_overall")
    fres_electron= TFile("acceptance/JPsi_electron_resolution_2d.root","READ")
    fres_positron = TFile("acceptance/JPsi_electron_resolution_2d.root","READ")
    fres_proton = TFile("acceptance/JPsi_proton_resolution_2d.root","READ")
    res_electron_p = fres_electron.Get("p_resolution")
    res_positron_p = fres_positron.Get("p_resolution")
    res_proton_p = fres_proton.Get("p_resolution")
    res_electron_th = fres_electron.Get("theta_resolution")
    res_positron_th = fres_positron.Get("theta_resolution")
    res_proton_th = fres_proton.Get("theta_resolution")
    res_electron_phi = fres_electron.Get("phi_resolution")
    res_positron_phi = fres_positron.Get("phi_resolution")
    res_proton_phi = fres_proton.Get("phi_resolution")
    res_electron_p.Scale(1.5)
    res_positron_p.Scale(1.5)
    res_proton_p.Scale(1.5)
    res_electron_th.Scale(1.5)
    res_positron_th.Scale(1.5)
    res_proton_th.Scale(1.5)
    res_electron_phi.Scale(1.5)
    res_positron_phi.Scale(1.5)
    res_proton_phi.Scale(1.5)
elif(detector=='CLAS12'):
    facc = TFile("acceptance/clasev_acceptance.root","r")
    acc = facc.Get("acceptance_PTheta_ele")

5961.2109375


## Generator

In [34]:
%%time
# Generate TFile
# ------------------------------
fname = "./data/{}_e{}_beamE_{:.2f}_evts_{}_process_{}_detector_{}_model_{}.root".format(fprefix,targetType,beamE,events,processStr,detector,modelType)
outFile=TFile(fname,"RECREATE")
# Generate TTree
# ------------------------------
outTree=TTree("tree","tree")
# Create the branches for the TTree
# ------------------------------
outTree.Branch("eOut",eOut)
outTree.Branch("hOut",hOut)
outTree.Branch("ePlus",ePlus)
outTree.Branch("eMinus",eMinus)
outTree.Branch("smear_eOut",smear_eOut)
outTree.Branch("smear_hOut",smear_hOut)
outTree.Branch("smear_ePlus",smear_ePlus)
outTree.Branch("smear_eMinus",smear_eMinus)
outTree.Branch("q",q)
outTree.Branch("smear_q",smear_q)
outTree.Branch("smear_VM",smear_VM)
outTree.Branch("weight",weight,"weight/D")
outTree.Branch("decay_weight",decay_weight,"decay_weight/D")
outTree.Branch("psf",psf,"psf/D")
outTree.Branch("flux",flux,"flux/D")
outTree.Branch("acc_eOut",acc_eOut,"acc_eOut/D")
outTree.Branch("acc_hOut",acc_hOut,"acc_hOut/D")
outTree.Branch("acc_ePlus",acc_ePlus,"acc_ePlus/D")
outTree.Branch("acc_eMinus",acc_eMinus,"acc_eMinus/D")
outTree.Branch("gammaE",gammaE_,"gammaE/D")
outTree.Branch("t",t_,"t/D")
outTree.Branch("m_vm",m_vm_,"m_vm/D")
# Print event information
# ------------------------------
print(detector," simulation")
print("--------------------------------------------------------")
print("Filename ==>",fname)
print("--------------------------------------------------------")
print("Target Type: ",targetType)
print("Target Length: ", d,"cm")
print("Radiation Length: ", X0, "cm")
if(processID==0):
    print("Beginning Monte Carlo Simulation of e({:.2f} GeV)+{} --> gamma + {} --> J/psi(e-e+) + {}'".format(beamE,targetType,targetType,targetType))
    print(" *** PHOTO-PRODUCTION *** ")
else:
    print("Beginning Monte Carlo Simulation of e({:.2f} GeV)+{} --> gamma* + {} --> e' + J/psi(e-e+) + {}'".format(beamE,targetType,targetType,targetType))
    print(" *** ELECTRO-PRODUCTION *** ")
print("--------------------------------------------------------")
print("Event Kinematics")
print(kmin, "< Egamma <", kmax,"[GeV]")
if(processID==0):
    print("EPA Maximum Q2 =",q2max,"[GeV^2]")
print("--------------------------------------------------------")
print("Number of Events =",events)
print("Luminosity =", np.round(lumi/3600.0/24/days,2),"(events/nb/s)")
print("Integrated Luminosity =",np.round(lumi,2),"(events/nb)")
print("--------------------------------------------------------\n\n")
success=0

for evt in range(events):
    
    # Print statement for updating user
    if( (evt+1)%(events/10)==0):
        print(evt+1,"out of",events)
        
    # Set Initial Vectors
    eIn.SetPxPyPzE(0,0,np.sqrt(beamE**2-mE**2),beamE)
    if(do_coherent==True):
        hIn.SetXYZM(0,0,0,mT)
    else:
        #hIn.SetXYZM(0,0,0,mT)
        p_rel,cth_rel,phi_rel = getNucleon()
        hIn.SetPxPyPzE(p_rel*np.sqrt(1-cth_rel**2)*np.cos(phi_rel),
                       p_rel*np.sqrt(1-cth_rel**2)*np.sin(phi_rel),
                       p_rel*cth_rel,
                       2*mT - np.sqrt(mT**2+p_rel**2))
    
    # Photon generation
    if(processID==0):
        # Do Bremmstrahlung & EPA
        flux[0], gammaE = N_Bremmstrahlung(kmin,kmax,beamE,d,X0)
        #flux[0]=flux[0]+N_EquivalentPhotonApproximation(gammaE,beamE,q2max)
        q.SetPxPyPzE(0,0,gammaE,gammaE)
    elif(processID==1):
        flux[0], temp = VirtualPhoton(kmin,kmax,beamE,mT)
        if(flux[0]==0):
            continue
        q.SetPxPyPzE(temp.Px(),temp.Py(),temp.Pz(),temp.E())
        gammaE=q.E()
    
    Q2 = -q*q
    # Get scattered electron
    # (Temp vector is necessary to avoid seg fault)
    temp = eIn-q
    eOut.SetPxPyPzE(temp.Px(),temp.Py(),temp.Pz(),temp.E())
    # Check W threshold
    W = (q+hIn).M()
    if(W < mJpsi+mT):
        continue
    
    # Decay (gamma + p,d) --> (J/psi + p,d)
    GenPhase.SetDecay(q+hIn,2,np.array([mJpsi,mT]))
    GenPhase.Generate()
    temp1, temp2 = GenPhase.GetDecay(0),GenPhase.GetDecay(1)
    VM.SetPxPyPzE(temp1.Px(),temp1.Py(),temp1.Pz(),temp1.E())
    hOut.SetPxPyPzE(temp2.Px(),temp2.Py(),temp2.Pz(),temp2.E())
    psf[0] = 4 * np.pi
    
    # Get event parameters
    t = (hIn-hOut).M2()
    pCM_Initial = np.sqrt((W**2-hIn*hIn-q*q)**2-4*(hIn*hIn)*(q*q))/(2*W)
    pCM_Final = np.sqrt((W**2-hOut*hOut-VM*VM)**2-4*(hOut*hOut)*(VM*VM))/(2*W)
    cth = (np.sqrt(q*q+pCM_Initial**2)*np.sqrt(VM*VM+pCM_Final**2)-q*VM)/(pCM_Initial*pCM_Final)
    # Get Cross Sectional Weight of Event
    weight[0] = get_xsec(targetType,mT,modelType,gammaE,W,t,cth)
    if(weight[0]<0):
        continue
    if(do_coherent==False and targetType=='d'):
        flux_rel = np.sqrt((q*hIn)**2-(q*q)*(hIn*hIn))/(mT*q.P())
        psf[0] *= flux_rel
   
    # Decay the J/Psi
    GenPhase.SetDecay(VM,2,np.array([mE,mE]))
    GenPhase.Generate()
    ep=GenPhase.GetDecay(0)
    em=GenPhase.GetDecay(1)
    ePlus.SetPxPyPzE(ep.Px(),ep.Py(),ep.Pz(),ep.E())
    eMinus.SetPxPyPzE(em.Px(),em.Py(),em.Pz(),em.E())
    

    # Get decay weight
    y = (eIn-eOut).E()/eIn.E()
    Q2 = - (eIn-eOut).M2()
    gy = np.sqrt(Q2) / beamE
    eps = (1.0 - y - 0.25 * gy * gy) / (1.0 - y + 0.5 * y * y + 0.25 * gy * gy)
    R = (1.0 + Q2/2.164/(mJpsi**2))**(2.131)-1.0
    r = eps * R / (1.0 + eps * R)
    Ep = VM*hOut/mJpsi
    p = np.sqrt(Ep**2-mT**2)
    l = np.sqrt((mJpsi**2-ePlus*ePlus-eMinus*eMinus)**2-4*(ePlus*ePlus)*(eMinus*eMinus))/(2*mJpsi)
    cth_decay = (Ep * mJpsi/2 - ePlus*hOut)/(p*l)
    wth = 0.75*(1.0 + r + (1.0 - 3.0 * r) * cth_decay**2)
    branch = 0.05971
    decay_weight[0]=wth*branch
    
    # Get acceptances of final state particles
    if(do_acc):
        acc_ePlus[0] = acc_e(ePlus)
        acc_eMinus[0]= acc_e(eMinus)
        acc_eOut[0]  = acc_e(eOut)
        acc_hOut[0]  = acc_h(hOut)
    gammaE_[0]   = gammaE
    t_[0]        = t
    m_vm_[0]     = VM.M()
    
    if(acc_eOut[0]>0 and do_smear):
        temp = smear_particle(eOut,"e-",detector)
        smear_eOut.SetPx(temp.Px())
        smear_eOut.SetPy(temp.Py())
        smear_eOut.SetPz(temp.Pz())
        smear_eOut.SetE(temp.E())
    if(acc_ePlus[0]>0 and do_smear):
        temp = smear_particle(ePlus,"e+",detector)
        smear_ePlus.SetPx(temp.Px())
        smear_ePlus.SetPy(temp.Py())
        smear_ePlus.SetPz(temp.Pz())
        smear_ePlus.SetE(temp.E())
    if(acc_eMinus[0]>0 and do_smear):
        temp = smear_particle(eMinus,"e-",detector)
        smear_eMinus.SetPx(temp.Px())
        smear_eMinus.SetPy(temp.Py())
        smear_eMinus.SetPz(temp.Pz())
        smear_eMinus.SetE(temp.E())
    if(acc_hOut[0]>0 and do_smear):
        temp = smear_particle(hOut,targetType,detector)
        smear_hOut.SetPx(temp.Px())
        smear_hOut.SetPy(temp.Py())
        smear_hOut.SetPz(temp.Pz())
        smear_hOut.SetE(temp.E())
        
    if(acc_ePlus[0] and acc_eMinus[0] and do_smear):
        temp = (smear_ePlus+smear_eMinus)
        smear_VM.SetPxPyPzE(temp.Px(),temp.Py(),temp.Pz(),temp.E())
    
    if(acc_ePlus[0] and acc_eMinus[0] and acc_hOut[0] and do_smear):
        temp = (smear_ePlus+smear_eMinus+smear_hOut-hIn)
        smear_q.SetPxPyPzE(temp.Px(),temp.Py(),temp.Pz(),temp.E())

    # Record if the event was successfully simulated
    if(weight[0]>0.0):
        success+=1
        outTree.Fill()
print(success)
outFile.Write()
outFile.Close()

SoLID  simulation
--------------------------------------------------------
Filename ==> ./data/test_py_ep_beamE_8.80_evts_5000000_process_photo_coherent_detector_SoLID_model_PomeronLQCD.root
--------------------------------------------------------
Target Type:  p
Target Length:  15 cm
Radiation Length:  890 cm
Beginning Monte Carlo Simulation of e(8.80 GeV)+p --> gamma + p --> J/psi(e-e+) + p'
 *** PHOTO-PRODUCTION *** 
--------------------------------------------------------
Event Kinematics
6 < Egamma < 8.8 [GeV]
EPA Maximum Q2 = 0.1 [GeV^2]
--------------------------------------------------------
Number of Events = 5000000
Luminosity = 5008.83 (events/nb/s)
Integrated Luminosity = 21638137500.0 (events/nb)
--------------------------------------------------------


500000 out of 5000000
1000000 out of 5000000
1500000 out of 5000000
2000000 out of 5000000
2500000 out of 5000000
3000000 out of 5000000
3500000 out of 5000000
4000000 out of 5000000
4500000 out of 5000000
5000000 out of 5